# Perceiver Experiments

In [ ]:
# Imports

from transformers import (
    PerceiverModel,
    PerceiverConfig,
)
from transformers.models.perceiver.modeling_perceiver import PerceiverBasicDecoder

from torch import nn, optim

from dataloader import xrd_dataloader

In [ ]:
# Set up model

config = PerceiverConfig(d_model=10000)
decoder = PerceiverBasicDecoder(
    config,
    num_channels=config.d_latents,
    output_num_channels=10000,
    final_project=True,
    trainable_position_encoding_kwargs={
        "num_channels": config.d_latents,
        "index_dims": 1,
    },
)
model = PerceiverModel(config, decoder=decoder)

In [ ]:
# Define loss

mse_loss = nn.MSELoss()

In [ ]:
def train_model(num_epochs=100):
    outputs = []
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    for epoch in range(num_epochs):
        for data in xrd_dataloader:
            data = data.float()
            # data = data.to(device)
            # ===================forward=====================
            output = model(inputs=data)
            loss = mse_loss(output.logits, data)
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))
        outputs.append((epoch, data, output))

In [ ]:
# Train the model

model.train(True)
train_model(num_epochs=1)
model.train(False)